In [ ]:
from fastai.tabular.all import *
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostRegressor
import lightgbm as lgb

In [ ]:
df_train = pd.read_csv('/kaggle/input/song-popularity-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/song-popularity-prediction/test.csv')

In [ ]:
df_train.drop('id', axis=1,inplace=True)
df_test.drop('id', axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
train_file = '/kaggle/input/song-popularity-prediction/train.csv'

In [ ]:
dep_var = 'song_popularity'

In [ ]:
cont,cat = cont_cat_split(df_train, dep_var=dep_var)

In [ ]:
cat

In [ ]:
cont

In [ ]:
# splits = RandomSplitter(valid_pct=0.0)(range_of(df_train))
# to = TabularPandas(df_train, y_block=CategoryBlock, procs=[Categorify, FillMissing,Normalize],
#                    cat_names = cat,
#                    cont_names = cont,
#                    y_names=dep_var,
#                    splits=splits)

In [ ]:
params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "binary",
    'subsample': 0.95312,
    'learning_rate': 0.0015,
    "max_depth": 3,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.7705303688019942,
    "min_child_samples": 290,
    "reg_alpha": 14.68267919457715,
    "reg_lambda": 66.156,
    "max_bin": 772,
    "min_data_per_group": 177,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 17,
    "verbosity": -1,
    'random_state':2022,
    'n_estimators':5100,
    'colsample_bytree':0.1107
    }
    
idx = list(range(len(df_train)))
# kf = KFold(n_splits=2, shuffle=True)
preds_fold = []
for _ in range(100):
    print(f'Fold {_}')
    random.shuffle(idx)
    train_index = idx[:int(0.8*len(df_train))]
    valid_index = idx[int(0.8*len(df_train)):]
    
    to = TabularPandas(df_train, procs=[Categorify,FillMissing,Normalize],
                       y_block = CategoryBlock(),
                       cont_names = cont,
                       cat_names = cat,
                       y_names=dep_var,
                       splits=(list(train_index), list(valid_index)))
    
    x_train = to.train.xs
    y_train = to.train.y
    
    x_valid = to.valid.xs
    y_valid = to.valid.ys
    
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_val = lgb.Dataset(x_valid, y_valid)
    
    model = lgb.train(params=params_lgb,
                  train_set=lgb_train,
                  valid_sets=lgb_val,
                  early_stopping_rounds=225,
                  verbose_eval=False)
        
    preds_valid = model.predict(x_valid)
    roc1= roc_auc_score(y_valid,preds_valid)
    print(f'ROC : {roc1}')
    
    to_new = to.train.new(df_test)
    to_new.process()
    preds_test = model.predict(to_new.xs)
    preds_fold.append(preds_test)

---------------------

In [ ]:
preds_fold = np.array(preds_fold)

In [ ]:
final_preds = np.mean(preds_fold, axis=0)

In [ ]:
submission = pd.read_csv('/kaggle/input/song-popularity-prediction/sample_submission.csv')

In [ ]:
submission['song_popularity'] = final_preds

In [ ]:
submission.to_csv('fastai_submission.csv', index=False)